In [ ]:
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
import cv2

In [ ]:
mydata = ".\\notMNIST_small"

In [ ]:
import os
root_folders=list()
for i in os.listdir(mydata):
    root_folders.append(os.path.join(mydata,i))
root_folders

In [ ]:
_img_Labels_file=list()
for i,v in enumerate(root_folders):
    print(i,v)
    if v :
        for file in os.listdir(v):
            try:
                file_path = os.path.join(v,file)
                print(file_path)
                img = cv2.imread(file_path,cv2.IMREAD_GRAYSCALE)
                resize_img = cv2.resize(img,(100,100))
                _img_Labels_file.append([i,resize_img])
            except Exception as e:
                pass
            

In [ ]:
# this is process to convert img to array 
# then we make it 1-D from 3-D
# then we resize according to our needs here 100 X 100.
img = ".\\notMNIST_small\\A\\MDEtMDEtMDAudHRm.png"
img = cv2.imread(img,cv2.IMREAD_GRAYSCALE)
resize_img = cv2.resize(img,(100,100))
height, width = resize_img.shape
print(height, width, resize_img.shape, type(resize_img) )
plt.imshow(img, cmap="binary")
plt.show()
# cv2.imshow("A",resize_img)

# The OpenCV waitKey() function is a required keyboard binding 
# function after imwshow()
# cv2.waitKey(0)

# destroy all windows command
# cv2.destroyAllWindows()

In [ ]:
# lets suffle data for randomly train to network.
import random
random.shuffle(_img_Labels_file)

In [ ]:
# lets just check data is suffle or not
for i in _img_Labels_file[10:20]:
    print(i)

In [ ]:
X =[]
Y =[]
for labels, features  in enumerate(_img_Labels_file):
#     print(labels , features)
    X.append(features[1])
    Y.append(features[0])



In [ ]:
print("Features ",X[160],"label ",Y[160]) 
# just we clarify weather our images will match with label or not.

plt.imshow(X[160],cmap="binary")
plt.show()

In [ ]:
# before we start doing our actual work firt we save our data
import pickle
pickle_out = open(".\\NotMnist_filterd_data\\X_features.pickle","wb")
pickle.dump(X, pickle_out)
pickle_out.close()

pickle_out = open(".\\NotMnist_filterd_data\\Y_labels.pickle","wb")
pickle.dump(Y, pickle_out)
pickle_out.close()


In [ ]:
# lets load again ... i'm just doing for good practice + save time while i need to play with it so no need to do again.
import pickle
X_features = pickle.load(open(".\\NotMnist_filterd_data\\X_features.pickle","rb"))
Y_labels = pickle.load(open(".\\NotMnist_filterd_data\\Y_labels.pickle","rb"))


In [ ]:
Y_labels[0]

In [ ]:
X_features = np.array(X_features).reshape(-1,100,100,1) 
# we need to pass again three args overhear also otherwise its just make it single vector array.
# Y_labels = np.array(Y_labels).reshape(-1,)
X_features.shape

In [ ]:
gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=0.5)
sess = tf.Session(config = tf.ConfigProto(gpu_options=gpu_options))
# normalization of data for easy to calculations.
X_features = X_features/255.0

In [ ]:
# i dont want to run and test now its take time to train the data.
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Conv2D,Activation,MaxPool2D,Dropout,Flatten
model = Sequential()
model.add(Conv2D(256,(3,3),input_shape=X_features.shape[1:]))
model.add(Activation('relu'))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Conv2D(256,(3,3)))
model.add(Activation('relu'))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Flatten())
model.add(Dense(1))
model.add(Activation('softmax'))
model.compile(loss="binary_crossentropy",optimizer="adam",metrics=['accuracy'])
model.fit(X_features,Y,batch_size=64,epochs=3,validation_split=0.3)